In [1]:
from Data import *

import pandas as pd
import string
import random

import torch
from torch import optim
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.utils.rnn import  pack_padded_sequence, pad_packed_sequence
from torchsummary import summary

import matplotlib.pyplot as plt
# plt.switch_backend('agg')
import numpy as np
import tqdm
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint_name = 'Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt'
loss_file = "Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout)_loss.npz"
print('Using {} device'.format(device))

Using cuda device


In [2]:
sp = 0
tp = 1
pg = 2
p = 3

def split_data(data):
    split_data = []
    for string in data:
        split_space = string.split()
        for i, word in enumerate(split_space):
            split_data.append(word)
    return split_data

def y_train_make(n):
    np_sp = np.array([sp])
    np_tp = np.array([tp])
    np_pg = np.array([pg])
    np_p = np.array([p])
    y = []
    for i in range(n):
        y.append(np_sp)
        y.append(np_tp)
        y.append(np_pg)
        y.append(np_p)
    return np.array(y)

def src_trg_split(data):
    src = []
    trg = []
    for i in range(0, len(data), 2):
        src.append(data[i])
        trg.append(data[i+1])
    return np.array(src), np.array(trg)

In [3]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

train_data = np.squeeze(pd.read_csv('train.txt', header=None))

y_train = y_train_make(len(train_data))

def to_one_hot(label):
    one_hot =  np.zeros((len(label), 4))
    one_hot[np.arange(len(label)), label[:, 0]] = 1
    return one_hot
    
# y_train = to_one_hot(y_train)
train_data = split_data(train_data)
train_loader = DataTransformer(train_data, y_train, use_cuda=True)

In [4]:
test_data = np.squeeze(pd.read_csv('test.txt', header=None))
test_data = split_data(test_data)
test_data = np.array(test_data)
src, trg = src_trg_split(test_data)
test_src = []
test_trg = []

for word in src:
    test_src.append(train_loader.vocab.sequence_to_indices(word, add_eos=True))

for word in trg:
    test_trg.append(train_loader.vocab.sequence_to_indices(word, add_eos=True))
"""
sp -> p
sp -> pg
sp -> tp
sp -> tp
p  -> tp
sp -> pg
p  -> sp
pg -> sp
pg -> p
pg -> tp
"""
test_c_src = np.array([sp, sp, sp, sp, p, sp, p, pg, pg, pg]).reshape(-1, 1)
test_c_trg = np.array([p, pg, tp, tp, tp, pg, sp, sp, p, tp]).reshape(-1, 1)
# test_c_src = Variable(torch.LongTensor(to_one_hot(test_c_src))).to(device)
test_c_src = Variable(torch.LongTensor(test_c_src)).to(device)
# test_c_trg = Variable(torch.LongTensor(to_one_hot(test_c_trg))).to(device)
test_c_trg = Variable(torch.LongTensor(test_c_trg)).to(device)

In [5]:
def test_eval(model, epoch):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():    
        src_c = test_src
        trg_c = test_trg
        optimizer.zero_grad()            
        output = model(src, trg, src_c, trg_c, 0) #turn off teacher forcing
        loss = criterion(output[1:].view(-1, output.shape[-1]), trg[1:].view(-1))
        
        if epoch % 10 == 0:
            print("=========show testing result=========")
            for i in range(output.shape[-1]):
                show_result(trg, output, i)
                print()
            
        return loss
    
def show_result(target, output, index):
    print("Ground true: ", end="")
    for i in range(max_len):
        tmp = trg[index, i]
        if tmp == 1:
            break
        elif tmp == 0:
            continue
        print(chr(trg[index, i]-7+ord('a')), end="")
    
    print(" Predict: ", end="")
    for i in range(max_len):
        print(chr(np.argmax(o[index, i, 3:])-4+ord('a')), end="")

In [6]:
class Encoder(nn.Module):

    def __init__(self, vocab_size, embedding_size, output_size, lat_dim):
        """Define layers for a vanilla rnn encoder"""
        super(Encoder, self).__init__()
        self.embedding_size = embedding_size
        
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.cond_embedding = nn.Embedding(4, 8)
        self.fc = nn.Linear(self.embedding_size+8, self.embedding_size)
        self.lstm = nn.LSTM(embedding_size, output_size)
        
        self.hidden2mean = nn.Linear(output_size, lat_dim)
        self.hidden2logv = nn.Linear(output_size, lat_dim)
        
        self.cell2mean = nn.Linear(output_size, lat_dim)
        self.cell2logv = nn.Linear(output_size, lat_dim)

    def forward(self, input_seqs, input_lengths, label, hidden=None, cell=None):
        embedded = self.embedding(input_seqs)
        packed = pack_padded_sequence(embedded, input_lengths)
        label = label.view(1, -1)
        label = self.cond_embedding(label)
        hidden = torch.cat([hidden, label], dim=2)
        hidden = self.fc(hidden)
        cell = hidden
        packed_outputs, (hidden, cell) = self.lstm(packed, (hidden, cell))
        outputs, output_lengths = pad_packed_sequence(packed_outputs)
        
        hidden_means = self.hidden2mean(hidden)
        hidden_logv = self.hidden2logv(hidden)
        
        cell_means = self.cell2mean(cell)
        cell_logv = self.cell2logv(cell)
        
        return outputs, hidden_means, hidden_logv, cell_means, cell_logv
    
    def initHidden(self, batch_size):
        return torch.zeros([1, batch_size, self.embedding_size]) # (D * num_layers, batch_size, H_out) 

In [7]:
class Decoder(nn.Module):

    def __init__(self, hidden_size, output_size, lat_dim, max_length, teacher_forcing_ratio, sos_id, use_cuda):
        """Define layers for a vanilla rnn decoder"""
        super(Decoder, self).__init__()

        self.word_dropout_rate = 0
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.cond_embedding = nn.Embedding(4, 8)
        self.latent2hidden = nn.Linear(lat_dim+8, hidden_size)
        self.latent2cell = nn.Linear(lat_dim+8, hidden_size)
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.log_softmax = nn.LogSoftmax(dim=-1)  # work with NLLLoss = CrossEntropyLoss

        self.max_length = max_length
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.sos_id = sos_id
        self.use_cuda = use_cuda

    def forward_step(self, inputs, hidden, cell):
        # inputs: (time_steps=1, batch_size)
        batch_size = inputs.size(1)
        embedded = self.embedding(inputs)
        embedded.view(1, batch_size, self.hidden_size)  # S = T(1) x B x N
        rnn_output, (hidden, cell) = self.lstm(embedded, (hidden, cell))  # S = T(1) x B x H
        rnn_output = rnn_output.squeeze(0)  # squeeze the time dimension
        output = self.log_softmax(self.out(rnn_output))  # S = B x O
        return output, hidden, cell

    def forward(self, context_vector, decoder_cell, targets):

        # Prepare variable for decoder on time_step_0
        target_vars, target_lengths = targets
        batch_size = context_vector.size(1)
        decoder_input = Variable(torch.LongTensor([[self.sos_id] * batch_size]))

        # Pass the context vector
        decoder_hidden = context_vector

        max_target_length = max(target_lengths)
        decoder_outputs = Variable(torch.zeros(
            max_target_length,
            batch_size,
            self.output_size
        ))  # (time_steps, batch_size, vocab_size)

        if self.use_cuda:
            decoder_input = decoder_input.cuda()
            decoder_outputs = decoder_outputs.cuda()
            
        if self.word_dropout_rate:
            prob = torch.rand(decoder_input.size()).to(device)
            prob[(decoder_input.data - train_loader.vocab.char2idx["SOS"]) * (decoder_input.data - train_loader.vocab.char2idx['PAD']) == 0] = 1
            decoder_input[prob < self.word_dropout_rate] = train_loader.vocab.char2idx['UNK']

        use_teacher_forcing = True if random.random() < self.teacher_forcing_ratio else False

        # Unfold the decoder RNN on the time dimension
        for t in range(max_target_length):
            decoder_outputs_on_t, decoder_hidden, decoder_cell = self.forward_step(decoder_input, decoder_hidden, decoder_cell)
            decoder_outputs[t] = decoder_outputs_on_t
            if use_teacher_forcing:
                decoder_input = target_vars[t].unsqueeze(0)
            else:
                decoder_input = self._decode_to_index(decoder_outputs_on_t)

        return decoder_outputs, decoder_hidden, decoder_cell

    def evaluate(self, context_vector, decoder_cell):
        batch_size = context_vector.size(1) # get the batch size
        decoder_input = Variable(torch.LongTensor([[self.sos_id] * batch_size]))
        decoder_hidden = context_vector

        decoder_outputs = Variable(torch.zeros(
            self.max_length,
            batch_size,
            self.output_size
        ))  # (time_steps, batch_size, vocab_size)

        if self.use_cuda:
            decoder_input = decoder_input.cuda()
            decoder_outputs = decoder_outputs.cuda()

        # Unfold the decoder RNN on the time dimension
        for t in range(self.max_length):
            decoder_outputs_on_t, decoder_hidden, decoder_cell = self.forward_step(decoder_input, decoder_hidden, decoder_cell)
            decoder_outputs[t] = decoder_outputs_on_t
            decoder_input = self._decode_to_index(decoder_outputs_on_t)  # select the former output as input

        return self._decode_to_indices(decoder_outputs)

    def _decode_to_index(self, decoder_output):
        """
        evaluate on the logits, get the index of top1
        :param decoder_output: S = B x V or T x V
        """
        value, index = torch.topk(decoder_output, 1)
        index = index.transpose(0, 1)  # S = 1 x B, 1 is the index of top1 class
        if self.use_cuda:
            index = index.cuda()
        return index

    def _decode_to_indices(self, decoder_outputs):
        """
        Evaluate on the decoder outputs(logits), find the top 1 indices.
        Please confirm that the model is on evaluation mode if dropout/batch_norm layers have been added
        :param decoder_outputs: the output sequence from decoder, shape = T x B x V 
        """
        decoded_indices = []
        batch_size = decoder_outputs.size(1)
        decoder_outputs = decoder_outputs.transpose(0, 1)  # S = B x T x V

        for b in range(batch_size):
            top_ids = self._decode_to_index(decoder_outputs[b])
            decoded_indices.append(top_ids.data[0].cpu().numpy())
        return decoded_indices

In [8]:
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, inputs, targets, label):
        # variables
        input_vars, input_lengths = inputs
        batch_size = input_vars.shape[1]
        encoder_hidden = self.encoder.initHidden(batch_size).to(device)
        encoder_cell = self.encoder.initHidden(batch_size).to(device)
        
        # encoder
        encoder_outputs, hidden_means, hidden_logv, cell_means, cell_logv = \
            self.encoder.forward(input_vars, input_lengths, label, hidden=encoder_hidden, cell=encoder_cell)  

        # reparaterization trick
        encoder_hidden = self.reparaterization_trick(hidden_means, hidden_logv)
        encoder_cell = self.reparaterization_trick(cell_means, cell_logv)
        label = label.view(1, -1)
        label = self.decoder.cond_embedding(label)
        encoder_hidden = torch.cat([encoder_hidden, label], dim=2)
        encoder_hidden = self.decoder.latent2hidden(encoder_hidden)
        encoder_cell = torch.cat([encoder_cell, label], dim=2)
        encoder_cell = self.decoder.latent2cell(encoder_cell)
        
        # decoder
        decoder_outputs, decoder_hidden, decoder_cell = self.decoder.forward(context_vector=encoder_hidden, 
                                                               decoder_cell=encoder_cell, targets=targets)
        
        return decoder_outputs, decoder_hidden, hidden_means, hidden_logv, cell_means, cell_logv

    def evaluate(self, inputs, src_label, trg_label):
        # variables
        input_vars, input_lengths = inputs
        batch_size = input_vars.shape[1]
        encoder_hidden = self.encoder.initHidden(batch_size).to(device)
        encoder_cell = self.encoder.initHidden(batch_size).to(device)
        
        # encoder
        encoder_outputs, hidden_means, hidden_logv, cell_means, cell_logv = \
            self.encoder.forward(input_vars, input_lengths, src_label, hidden=encoder_hidden, cell=encoder_cell)  

        # reparaterization trick
        encoder_hidden = self.reparaterization_trick(hidden_means, hidden_logv)
        encoder_cell = self.reparaterization_trick(cell_means, cell_logv)
        trg_label = trg_label.view(1, -1)
        trg_label = self.decoder.cond_embedding(trg_label)
        encoder_hidden = torch.cat([encoder_hidden, trg_label], dim=2)
        encoder_hidden = self.decoder.latent2hidden(encoder_hidden)
        encoder_cell = torch.cat([encoder_cell, trg_label], dim=2)
        encoder_cell = self.decoder.latent2cell(encoder_cell)
        
        # decoder
        decoded_sentence = self.decoder.evaluate(context_vector=encoder_hidden, decoder_cell=encoder_cell)
        
        return decoded_sentence
    
    def reparaterization_trick(self, mean, logv):
        std = torch.exp(0.5*logv)
        eps = torch.randn_like(std)
        return  mean + eps * std

In [9]:
class Trainer(object):

    def __init__(self, model, data_transformer, label, learning_rate, use_cuda, checkpoint_name,
                 teacher_forcing_ratio=1.0, kl_weight=0, word_dropout_rate=0):

        self.model = model
        self.checkpoint_name = checkpoint_name
        self.total_iter = 0
        
        # save list
        self.entropy = []
        self.kld = []
        self.kl_weight_list = []
        self.teacher_forcing_ratio_list = []
        self.score = []
        
        # init hyperparameters
        self.kl_weight = kl_weight
        self.model.decoder.word_dropout_rate = word_dropout_rate
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.kl_weight_list.append(self.kl_weight)
        self.teacher_forcing_ratio_list.append(self.teacher_forcing_ratio)
        
        # record some information about dataset
        self.data_transformer = data_transformer
        self.label = label
        self.vocab_size = self.data_transformer.vocab_size
        self.PAD_ID = self.data_transformer.PAD_ID
        self.use_cuda = use_cuda
        
        # optimizer setting
        self.learning_rate = learning_rate
        self.optimizer= torch.optim.Adam(self.model.parameters(), lr=learning_rate)
#         self.optimizer= torch.optim.SGD(self.model.parameters(), lr=learning_rate)
        self.criterion = torch.nn.NLLLoss(ignore_index=self.PAD_ID, reduction='mean')

    def train(self, num_epochs, batch_size, pretrained=False):

        if pretrained:
            self.load_model()
        
        self.model.train()
        step = 0

        for epoch in range(0, num_epochs):
            mini_batches = self.data_transformer.mini_batches(batch_size=batch_size)
            for input_batch, target_batch, label_batch in mini_batches:
                self.total_iter += 1
                self.optimizer.zero_grad()
                self.model.decoder.teacher_forcing_ratio = self.teacher_forcing_ratio
                decoder_outputs, decoder_hidden, hidden_means, hidden_logv, cell_means, cell_logv = \
                    self.model(input_batch, target_batch, label_batch)

                # calculate the loss and back prop.
                cur_loss = self.get_loss(decoder_outputs, target_batch[0])
                kl_loss = self.kl_weight * self.get_kl_loss(hidden_means, hidden_logv)+\
                            self.kl_weight* self.get_kl_loss(cell_means, cell_logv)
                loss = cur_loss + kl_loss
                
                self.entropy.append(cur_loss.item())
                self.kld.append(kl_loss.item())
                
                # logging
                step += 1
                if step % 50 == 0:
                    print("Step:", step, "char-loss: ", loss.item())
                    print("KL_weight: ", self.kl_weight, "teacher_forcing_ratio: ", self.teacher_forcing_ratio)
                    self.save_model()
                loss.backward()

                # optimize
                self.optimizer.step()
                
                # update hyperparameters
                self.kl_weight = self.get_kl_weight(self.kl_weight)
                self.model.decoder.word_dropout_rate = self.get_word_dropout_rate(self.model.decoder.word_dropout_rate)
                self.teacher_forcing_ratio = self.get_teacher_forcing_ratio(self.teacher_forcing_ratio)
                self.kl_weight_list.append(self.kl_weight)
                self.teacher_forcing_ratio_list.append(self.teacher_forcing_ratio)

        self.save_model()

    def get_loss(self, decoder_outputs, targets):
        b = decoder_outputs.size(1)
        t = decoder_outputs.size(0)
        targets = targets.contiguous().view(-1)  # S = (B*T)
        decoder_outputs = decoder_outputs.view(b * t, -1)  # S = (B*T) x V
        return self.criterion(decoder_outputs, targets)
    
    def get_kl_loss(self, mean, logvar):
        result = -0.5 * torch.sum(logvar - torch.pow(mean, 2) - torch.exp(logvar) + 1, 1)
        return result.mean()
    
    def get_kl_weight(self, kl_weight):
#         return 0
#         if self.total_iter < 50000:
#             return 0
#         else:
#         return 0
        return min(1, kl_weight + 0.0000001)

    def get_word_dropout_rate(self, word_dropout_rate):
#         if self.total_iter < 50000:
#             return 0
#         else:
#         return 0
        return 0
#         return min(1, word_dropout_rate + 0.000001)

    def get_teacher_forcing_ratio(self, teacher_forcing_ratio):
#         if self.total_iter < 100000:
#             return 1
#         else:
        return max(0, teacher_forcing_ratio - 0.00000001)
#         return 1
    
    def save_model(self):
        torch.save(self.model.state_dict(), self.checkpoint_name)
        np.savez(loss_file, entropy=self.entropy, kld=self.kld, kl_weight=self.kl_weight_list,\
                 teacher_forcing_ratio=self.teacher_forcing_ratio_list, score=self.score)
        print("Model has been saved as %s.\n" % self.checkpoint_name)

    def load_model(self):
        self.model.load_state_dict(torch.load(self.checkpoint_name, map_location=device))
        load_file = np.load(loss_file)
        self.entropy = load_file['entropy'].tolist()
        self.kld = load_file['kld'].tolist()
        self.kl_weight_list = load_file['kl_weight'].tolist()
        self.teacher_forcing_ratio_list = load_file['teacher_forcing_ratio'].tolist()
        self.score = load_file['score'].tolist()
        print("Pretrained model has been loaded.\n")

    def evaluate(self, words, src_label, trg_label):
        # make sure that words is list
        if type(words) is not list:
            words = [words]
        self.model.eval()
        # transform word to index-sequence
        eval_var = self.data_transformer.evaluation_batch(words=words)
        decoded_indices = self.model.evaluate(eval_var, src_label, trg_label)
        results = []
        for indices in decoded_indices:
            results.append(self.data_transformer.vocab.indices_to_sequence(indices))
        return results
    
    def compute_bleu(self, output, reference):
        cc = SmoothingFunction()
        if len(reference) == 3:
            weights = (0.33,0.33,0.33)
        else:
            weights = (0.25,0.25,0.25,0.25)
        return sentence_bleu([reference], output,weights=weights,smoothing_function=cc.method1)

In [10]:
# define our models
encoder = Encoder(vocab_size=train_loader.vocab_size,
                         embedding_size=256,
                         output_size=256,
                         lat_dim=32).to(device)

decoder = Decoder(hidden_size=256,
                         output_size=train_loader.vocab_size,
                         lat_dim=32,
                         max_length=train_loader.max_length,
                         teacher_forcing_ratio=1.,
                         sos_id=train_loader.SOS_ID,
                         use_cuda=True).to(device)

seq2seq = Seq2Seq(encoder=encoder,
                  decoder=decoder)

trainer = Trainer(seq2seq, train_loader, y_train, learning_rate=0.001, use_cuda=True, checkpoint_name=checkpoint_name, 
                        teacher_forcing_ratio=1, kl_weight=0, word_dropout_rate=0)

In [11]:
for epoch in tqdm.notebook.tqdm(range(100)):
    trainer.train(num_epochs=10, batch_size=128, pretrained=False)
#     torch.save(seq2seq.state_dict(), "Lab4_seq2seq_vae_lstm_KL_cond_(word_dropout)_{}epoch.pt".format(epoch))

    ## eval
    print("========================================Evaluating========================================")  
    total_score = 0.0
    for i in range(len(test_src)):
        word = train_loader.vocab.indices_to_sequence(test_src[i])
        trg_true = train_loader.vocab.indices_to_sequence(test_trg[i])
        results = trainer.evaluate(word, test_c_src[i].view(1, -1), test_c_trg[i].view(1, -1))[0]
        score = trainer.compute_bleu(results, trg_true)
        print("Src_true: {:>12}".format(word), "\tTrg_true:{:>12}".format(trg_true), "\tPredict: {:>12}".format(results), "\tScore: {:>8.5f}".format(score))
        total_score += score
    total_score /= len(test_src)
    trainer.score.append(total_score)
    print("==========================================================================================\n")

Step: 50 char-loss:  1.9408141374588013
KL_weight:  4.899999999999996e-06 teacher_forcing_ratio:  0.9999995099999975
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 100 char-loss:  1.344679355621338
KL_weight:  9.90000000000001e-06 teacher_forcing_ratio:  0.999999009999995
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 150 char-loss:  0.791664183139801
KL_weight:  1.4900000000000047e-05 teacher_forcing_ratio:  0.9999985099999925
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 200 char-loss:  0.4624713659286499
KL_weight:  1.9900000000000006e-05 teacher_forcing_ratio:  0.99999800999999
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 250 char-loss:  0.29705867171287537
KL_weight:  2.4899999999999958e-05 teacher_forcing_ratio:  0.9999975099999875
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 200 char-loss:  0.06158626079559326
KL_weight:  0.0001369000000000025 teacher_forcing_ratio:  0.9999863099999312
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 250 char-loss:  0.10779936611652374
KL_weight:  0.0001419000000000026 teacher_forcing_ratio:  0.9999858099999287
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 300 char-loss:  0.06035707890987396
KL_weight:  0.00014690000000000273 teacher_forcing_ratio:  0.9999853099999262
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 350 char-loss:  0.05543780326843262
KL_weight:  0.00015190000000000285 teacher_forcing_ratio:  0.9999848099999237
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

========================================Evaluating========================================
Src_true:      a

Step: 350 char-loss:  0.06450389325618744
KL_weight:  0.00026890000000000567 teacher_forcing_ratio:  0.9999731099998649
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

========================================Evaluating========================================
Src_true:      abandon 	Trg_true:   abandoned 	Predict:   anforathed 	Score:  0.06985
Src_true:         abet 	Trg_true:    abetting 	Predict: bestallening 	Score:  0.11531
Src_true:        begin 	Trg_true:      begins 	Predict:   begreakins 	Score:  0.17567
Src_true:       expend 	Trg_true:     expends 	Predict: expenscertes 	Score:  0.33181
Src_true:         sent 	Trg_true:       sends 	Predict:  sentelshing 	Score:  0.10025
Src_true:        split 	Trg_true:   splitting 	Predict: splestrifling 	Score:  0.14694
Src_true:       flared 	Trg_true:       flare 	Predict:    fleracuse 	Score:  0.06377
Src_true:  functioni

Step: 50 char-loss:  0.07407566159963608
KL_weight:  0.00039490000000000873 teacher_forcing_ratio:  0.9999605099998016
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 100 char-loss:  0.07067793607711792
KL_weight:  0.00039990000000000885 teacher_forcing_ratio:  0.9999600099997991
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 150 char-loss:  0.07284863293170929
KL_weight:  0.000404900000000009 teacher_forcing_ratio:  0.9999595099997965
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 200 char-loss:  0.08126019686460495
KL_weight:  0.0004099000000000091 teacher_forcing_ratio:  0.999959009999794
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 250 char-loss:  0.0780143067240715
KL_weight:  0.0004149000000000092 teacher_forcing_ratio:  0.9999585099997915
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropou

Step: 200 char-loss:  0.07791982591152191
KL_weight:  0.000526899999999991 teacher_forcing_ratio:  0.9999473099997352
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 250 char-loss:  0.08527127653360367
KL_weight:  0.0005318999999999884 teacher_forcing_ratio:  0.9999468099997327
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 300 char-loss:  0.08481226861476898
KL_weight:  0.0005368999999999858 teacher_forcing_ratio:  0.9999463099997302
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 350 char-loss:  0.08449311554431915
KL_weight:  0.0005418999999999832 teacher_forcing_ratio:  0.9999458099997277
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

========================================Evaluating========================================
Src_true:      aban

...

Step: 200 char-loss:  0.15650811791419983
KL_weight:  0.002086900000000097 teacher_forcing_ratio:  0.9997913099989514
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 250 char-loss:  0.15002137422561646
KL_weight:  0.002091900000000089 teacher_forcing_ratio:  0.9997908099989489
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 300 char-loss:  0.1515604555606842
KL_weight:  0.002096900000000081 teacher_forcing_ratio:  0.9997903099989464
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 350 char-loss:  0.15543662011623383
KL_weight:  0.002101900000000073 teacher_forcing_ratio:  0.9997898099989438
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

========================================Evaluating========================================
Src_true:      abandon 

Step: 350 char-loss:  0.1569172441959381
KL_weight:  0.0022188999999998857 teacher_forcing_ratio:  0.9997781099988851
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

========================================Evaluating========================================
Src_true:      abandon 	Trg_true:   abandoned 	Predict:    abounched 	Score:  0.07937
Src_true:         abet 	Trg_true:    abetting 	Predict:   botholding 	Score:  0.11868
Src_true:        begin 	Trg_true:      begins 	Predict:    befittens 	Score:  0.07583
Src_true:       expend 	Trg_true:     expends 	Predict:    endoubles 	Score:  0.13485
Src_true:         sent 	Trg_true:       sends 	Predict:   smenalizes 	Score:  0.05308
Src_true:        split 	Trg_true:   splitting 	Predict:    sluggling 	Score:  0.14924
Src_true:       flared 	Trg_true:       flare 	Predict:       forgus 	Score:  0.04855
Src_true:  functioning 

Step: 50 char-loss:  0.16068397462368011
KL_weight:  0.002344899999999684 teacher_forcing_ratio:  0.9997655099988217
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 100 char-loss:  0.15070797502994537
KL_weight:  0.002349899999999676 teacher_forcing_ratio:  0.9997650099988192
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 150 char-loss:  0.152724027633667
KL_weight:  0.002354899999999668 teacher_forcing_ratio:  0.9997645099988167
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 200 char-loss:  0.15660682320594788
KL_weight:  0.00235989999999966 teacher_forcing_ratio:  0.9997640099988142
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 250 char-loss:  0.15678946673870087
KL_weight:  0.002364899999999652 teacher_forcing_ratio:  0.9997635099988117
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.


Step: 200 char-loss:  0.15597432851791382
KL_weight:  0.0024768999999994724 teacher_forcing_ratio:  0.9997523099987554
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 250 char-loss:  0.1599881947040558
KL_weight:  0.0024818999999994644 teacher_forcing_ratio:  0.9997518099987529
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 300 char-loss:  0.16618743538856506
KL_weight:  0.0024868999999994564 teacher_forcing_ratio:  0.9997513099987504
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Step: 350 char-loss:  0.14972318708896637
KL_weight:  0.0024918999999994484 teacher_forcing_ratio:  0.9997508099987479
Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

Model has been saved as Lab4_seq2seq_vae_lstm_KL_condembedding_(word_dropout).pt.

========================================Evaluating========================================
Src_true:      aban

KeyboardInterrupt: 

In [ ]:
plt.figure()

load = True
if load:
    acc = np.load(loss_file)
    entropy = acc['entropy']
    kld = acc['kld']
    kl_weight = acc['kl_weight']
    teacher_forcing_ratio = acc['teacher_forcing_ratio']
    score = acc['score']
    
# plt.title("Activation Function comparision(EEGNet)")
plt.ylabel("Loss")
plt.xlabel("Step")
# print(test_loss)
x = np.arange(1, len(entropy)+1)
plt.plot(x, entropy, label="CrossEntropy")
plt.plot(x, kld, label="kld")
plt.legend()
plt.show()
plt.clf()

plt.ylabel("KL_weight & teacher_forcing_ratio")
plt.xlabel("Step")
# print(test_loss)
x = np.arange(1, len(kl_weight)+1)
plt.plot(x, kl_weight, label="kl_weight")
plt.plot(x, teacher_forcing_ratio, label="teacher_forcing_ratio")
# plt.plot(x, test_loss, label="test")
plt.legend()
plt.show()

plt.ylabel("Score")
plt.xlabel("Step")
# print(test_loss)
x = np.linspace(1, len(kl_weight)+1, len(score))
plt.plot(x, score, label="score")
# plt.plot(x, test_loss, label="test")
plt.legend()
plt.show()


In [ ]:
def reparaterization_trick(mean, logv):
        std = torch.exp(0.5*logv)
        eps = torch.randn_like(std)
        return  mean + eps * std
    
def Gaussian_score(words):
    words_list = []
    score = 0
    yourpath = './train.txt'#should be your directory of train.txt
    with open(yourpath,'r') as fp:
        for line in fp:
            word = line.split(' ')
            word[3] = word[3].strip('\n')
            words_list.extend([word])
        for t in words:
            for i in words_list:
                if t == i:
                    score += 1
    return score/len(words)

    
label = torch.LongTensor([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]).to(device)

decoder.eval()
words = []
for i in range(100):        

    hidden_mean = torch.randn([1, 1, 32]).to(device)
    hidden_logv = torch.randn([1, 1, 32]).to(device)
    cell_mean = torch.randn([1, 1, 32]).to(device)
    cell_logv = torch.randn([1, 1, 32]).to(device)

    encoder_hidden = reparaterization_trick(hidden_mean, hidden_logv)
    encoder_hidden = decoder.latent2hidden(encoder_hidden)
    encoder_cell = reparaterization_trick(cell_mean, cell_logv)
    encoder_cell = decoder.latent2hidden(encoder_cell)
    
    tmp = []
    for i in range(4):
        hidden = torch.cat([encoder_hidden, label[i].view(1, 1, 4)], dim=2)
        cell = torch.cat([encoder_cell, label[i].view(1, 1, 4)], dim=2)
        decoded_indices = decoder.evaluate(context_vector=hidden, decoder_cell=cell)
        results = []
        for indices in decoded_indices:
            results.append(train_loader.vocab.indices_to_sequence(indices))
        tmp.append(results[0])
    words.append(tmp)
print(words)
print(Gaussian_score(words))

In [ ]:
total_score = 0.0
for i in range(len(test_src)):
    word = train_loader.vocab.indices_to_sequence(test_src[i])
    trg_true = train_loader.vocab.indices_to_sequence(test_trg[i])
    results = trainer.evaluate(word, test_c_src[i].view(1, -1), test_c_trg[i].view(1, -1))[0]
    score = trainer.compute_bleu(results, trg_true)
    print("Src_true: {:>12}".format(word), "\tTrg_true:{:>12}".format(trg_true), "\tPredict: {:>12}".format(results), "\tScore: {:>8.5f}".format(score))
    total_score += score
total_score /= len(test_src)
print(total_score)

In [ ]:
torch.save(decoder, "lstm2_decoder.pt")